In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

In [3]:
train = pd.read_csv('hw3_train.csv')
print(train.shape)
y = train['use_direct_debit']
train = train.drop('use_direct_debit', axis=1)
print(train.shape)

(7500, 29)
(7500, 28)


In [4]:
test = pd.read_csv('hw3_test_data.csv')
print(test.shape)

(7500, 28)


In [5]:
# Стандартный подход: берем модель (дерево), обучаем, получаем предсказания

dtc = DecisionTreeClassifier()
dtc.fit(train, y)
pred = dtc.predict_proba(test)[:,1] 

# Дает качество 0.687

In [6]:
roc_auc_score(y, pred)

0.573297302763832

In [7]:
# Попробуем сделать стекинг

dtc = DecisionTreeClassifier()
lr = LogisticRegression()

# Обучаем логрег и ее предсказания записываем в отдельную фичу
lr.fit(train, y)
train['new_stacking_feature_train'] = lr.predict_proba(train)[:,1] 
test['new_stacking_feature_test']   = lr.predict_proba(test)[:,1] 

# На всех данных с новой фичой обучаем новую модель (дерево) и делаем предсказание
dtc.fit(train, y)
pred = dtc.predict_proba(test)[:,1] 

# Дает качество 0.683

/home/jupyterhub/jupyterhub_venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [8]:
roc_auc_score(y, pred)

0.5683830407799558

In [9]:
# Стало хуже
# Есть проблема: ваша задача найти проблему и ее исправить

In [10]:
# Новые фичи не создаем, другие модели не используем
# Дело не в преобразовании старых фичей или в используемых моделях (дереве и лог-рег-е)

### Реализуем обучение ансамбля правильно, как это описано в лекции

Разобьем обучающую выборку на три множества $A, B$ и $C$:

In [160]:
from sklearn.model_selection import train_test_split

X_train_A, X_train_C, y_train_A, y_train_C = train_test_split(
        train, y, test_size=0.4, random_state=0)

In [161]:
X_train_A.shape, X_train_B.shape, 

((4500, 29), (1238, 29))

In [162]:
X_train_A, X_train_B, y_train_A, y_train_B = train_test_split(
        X_train_A, y_train_A, test_size=0.5, random_state=0)

In [163]:
X_train_A.shape, X_train_B.shape, X_train_C.shape

((2250, 29), (2250, 29), (3000, 29))

In [164]:
X_train_A.head()

,customer_id,customer_residence_code,gender,age,is_new_customer,seniority_month,customer_type,customer_relation_type,residence_same_as_bank,country_same_as_bank,...,use_payroll_account,use_short_deposits,use_medium_deposits,use_long_deposits,use_e_account,use_loans,use_taxes,use_credit_card,use_pensions,new_stacking_feature_train
6419,225370,1,0,23,0.0,35,1.0,0,2,0,...,0,0,0,0,0,0,0,0,0.0,0.110358
7426,327441,1,0,23,0.0,36,1.0,0,2,0,...,0,0,0,0,0,0,0,0,0.0,0.120518
4128,338589,1,0,39,0.0,37,1.0,1,2,0,...,0,0,0,0,1,0,0,0,0.0,0.181329
3206,202995,1,0,39,0.0,38,1.0,1,2,0,...,0,0,0,0,0,0,0,0,0.0,0.107740
4274,302722,1,0,15,0.0,37,1.0,1,2,0,...,0,0,0,0,0,0,0,0,0.0,0.121332


In [165]:
X_train_B.head()

,customer_id,customer_residence_code,gender,age,is_new_customer,seniority_month,customer_type,customer_relation_type,residence_same_as_bank,country_same_as_bank,...,use_payroll_account,use_short_deposits,use_medium_deposits,use_long_deposits,use_e_account,use_loans,use_taxes,use_credit_card,use_pensions,new_stacking_feature_train
3177,200887,1,0,25,0.0,38,1.0,1,2,2,...,0,0,0,0,0,0,0,0,0.0,0.094049
2668,292368,1,2,50,0.0,38,1.0,0,2,0,...,0,0,0,0,1,0,0,0,0.0,0.150128
7435,328030,1,2,31,0.0,36,1.0,1,2,0,...,0,0,0,0,0,0,0,0,0.0,0.128463
6868,181133,1,2,25,0.0,35,1.0,1,2,0,...,0,0,0,0,0,0,0,0,0.0,0.104087
701,93980,1,0,23,0.0,36,1.0,0,2,0,...,0,0,0,0,0,0,0,0,0.0,0.085792


In [166]:
X_train_C.head()

,customer_id,customer_residence_code,gender,age,is_new_customer,seniority_month,customer_type,customer_relation_type,residence_same_as_bank,country_same_as_bank,...,use_payroll_account,use_short_deposits,use_medium_deposits,use_long_deposits,use_e_account,use_loans,use_taxes,use_credit_card,use_pensions,new_stacking_feature_train
1563,226908,1,0,23,0.0,36,1.0,1,2,0,...,0,0,0,0,0,0,0,0,0.0,0.104084
5333,58838,1,2,23,0.0,36,1.0,1,2,0,...,0,0,0,0,0,0,0,0,0.0,0.081682
7424,326573,1,2,23,0.0,36,1.0,1,2,0,...,0,0,0,0,0,0,0,0,0.0,0.119449
6731,151683,1,2,23,0.0,35,1.0,1,2,0,...,0,0,0,0,0,0,0,0,0.0,0.097991
4178,332327,1,2,39,0.0,37,1.0,1,2,2,...,0,0,0,0,0,0,0,0,0.0,0.151123


Учим базовую модель $h_i(x)$ - в данном случае одну логистическую регрессию - на множестве $А$, и предсказываем - на $B$ и $C$

In [167]:
lr = LogisticRegression()
lr.fit(X_train_A, y_train_A)
pred_B = lr.predict_proba(X_train_B)[:,1]
pred_C = lr.predict_proba(X_train_C)[:,1] 

/home/jupyterhub/jupyterhub_venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


На предсказаниях для $B$ обучаем модель второго уровня (решающее правило $d(h_i(x))$ - в данном случае дерево решений с новой фичой с предсказанием от логистической регрессии:

In [168]:
dtc = DecisionTreeClassifier()
X_train_B['new_stacking_feature_train'] = pred_B

dtc.fit(X_train_B, y_train_B);

Используем $d$ для получения финальных предсказаний для $C$:

In [169]:
X_train_C['new_stacking_feature_train'] = pred_C
pred_final = dtc.predict_proba(X_train_C)[:,1] 

/home/jupyterhub/jupyterhub_venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [170]:
# стало
new_score = roc_auc_score(y_train_C, pred_final)
new_score

0.6931600947842063

In [171]:
# было
old_score = roc_auc_score(y, pred)
old_score

0.5683830407799558

Видим, что от стекинга и правильной валидации на обучающей выборке score увеличился на:

In [172]:
abs(new_score-old_score)

0.12477705400425043